In [1]:
!curl -sSOL https://downloads.apache.org/kafka/3.6.0/kafka_2.13-3.6.0.tgz
!tar -xzf kafka_2.13-3.6.0.tgz

In [2]:
!./kafka

/bin/bash: line 1: ./kafka: No such file or directory


In [3]:
!./kafka_2.13-3.6.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.6.0/config/zookeeper.properties
!./kafka_2.13-3.6.0/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.6.0/config/server.properties

In [4]:
!pip install pyspark
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=0f36e8ca2d04c51581cf77e0ff61a4e0310b1d45084640f04715d5e6e6683e9a
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 4.9 MB/s eta 0:00:00


In [7]:
import pyspark
from pyspark.sql import SparkSession

scala_version = '2.12'
spark_version = '3.5.0'
packages = [ f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}' ,
                                                'org.apache.kafka:kafka-clients:3.6.0']
spark = SparkSession.builder.master("local")\
                            .appName("kafka-example")\
                            .config("spark.jars.packages", ",".join(packages))\
                            .getOrCreate()
spark


In [8]:
from kafka import KafkaProducer
from json import dumps
topic_name = 'Number'
kafka_server = 'localhost:9092'
producer = KafkaProducer(bootstrap_servers=kafka_server,
                         value_serializer = lambda x:dumps(x).encode('utf-8'))
for e in range(1000):
    data = {'number' : e}
    producer.send(topic_name, value=data)

producer.flush()

kafkaDf = spark.read.format("kafka")\
                    .option("kafka.bootstrap.servers", "localhost:9092")\
                    .option("subscribe", topic_name)\
                    .option("startingOffsets", "earliest")\
                    .load()

kafkaDf.show()

+----+--------------------+------+---------+------+--------------------+-------------+
| key|               value| topic|partition|offset|           timestamp|timestampType|
+----+--------------------+------+---------+------+--------------------+-------------+
|NULL|[7B 22 6E 75 6D 6...|Number|        0|     0|2023-11-23 04:53:...|            0|
|NULL|[7B 22 6E 75 6D 6...|Number|        0|     1|2023-11-23 04:53:...|            0|
|NULL|[7B 22 6E 75 6D 6...|Number|        0|     2|2023-11-23 04:53:...|            0|
|NULL|[7B 22 6E 75 6D 6...|Number|        0|     3|2023-11-23 04:53:...|            0|
|NULL|[7B 22 6E 75 6D 6...|Number|        0|     4|2023-11-23 04:53:...|            0|
|NULL|[7B 22 6E 75 6D 6...|Number|        0|     5|2023-11-23 04:53:...|            0|
|NULL|[7B 22 6E 75 6D 6...|Number|        0|     6|2023-11-23 04:53:...|            0|
|NULL|[7B 22 6E 75 6D 6...|Number|        0|     7|2023-11-23 04:53:...|            0|
|NULL|[7B 22 6E 75 6D 6...|Number|        0

In [ ]:
!./kafka_2.13-3.6.0/bin/kafka-console-consumer.sh --topic Number --from-beginning --bootstrap-server 127.0.0.1:9092

In [9]:
# • Show the dataframe in a formatted way
from pyspark.sql.functions import col, concat, lit
kafkaDf.select(
    concat(col("topic"), lit(':'),
    col("partition").cast("string")).alias("topic_partition"),
    col("offset"),
    col("value").cast("string")
    ).show()

+---------------+------+--------------+
|topic_partition|offset|         value|
+---------------+------+--------------+
|       Number:0|     0| {"number": 0}|
|       Number:0|     1| {"number": 1}|
|       Number:0|     2| {"number": 2}|
|       Number:0|     3| {"number": 3}|
|       Number:0|     4| {"number": 4}|
|       Number:0|     5| {"number": 5}|
|       Number:0|     6| {"number": 6}|
|       Number:0|     7| {"number": 7}|
|       Number:0|     8| {"number": 8}|
|       Number:0|     9| {"number": 9}|
|       Number:0|    10|{"number": 10}|
|       Number:0|    11|{"number": 11}|
|       Number:0|    12|{"number": 12}|
|       Number:0|    13|{"number": 13}|
|       Number:0|    14|{"number": 14}|
|       Number:0|    15|{"number": 15}|
|       Number:0|    16|{"number": 16}|
|       Number:0|    17|{"number": 17}|
|       Number:0|    18|{"number": 18}|
|       Number:0|    19|{"number": 19}|
+---------------+------+--------------+
only showing top 20 rows

